## Reinforcement Learning Models

### Imports and Utils

In [1]:
"""
Importing necessary libraries
"""
import pygame
import neat
import os
import random

import pickle

pygame 2.4.0 (SDL 2.26.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Defining Utilities for the Game
"""
scale = 1.5
WIN_WIDTH = 288 * scale
WIN_HEIGHT = 512 * scale

def Load_Images(path : str, scale : int = 1):
    """
    path: str: path to the image file
    scale: int: scale of the image

    Returns: pygame.Surface: image loaded and scaled
    """
    img = pygame.image.load(path)
    return pygame.transform.scale(img, (int(img.get_width() * scale), int(img.get_height() * scale)))

BIRD_IMGS = [Load_Images(os.path.join("Images", "bird1.png"), scale), Load_Images(os.path.join("Images", "bird2.png"), scale), Load_Images(os.path.join("Images", "bird3.png"), scale)]
PIPE_IMG = Load_Images(os.path.join("Images", "pipe.png"), scale)
BASE_IMG = Load_Images(os.path.join("Images", "base.png"), scale)
BG_IMG = Load_Images(os.path.join("Images", "bg.png"), scale)

In [3]:
class Bird:
    """
    The Bird Class
    """
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25
    ROT_VEL = 10
    ANIMATION_TIME = 5

    def __init__(self, x, y):
        """
        A Constructor for the Bird Class

        x: int: x-coordinate of the bird
        y: int: y-coordinate of the bird
        """

        self.x = x
        self.y = y
        self.tilt = 0
        self.vel = 0
        self.cnt = 0
        self.img = self.IMGS[0]

    def jump(self):
        """
        A function to make the bird jump
        """

        self.vel = -20
        self.height = self.y

    def move(self):
        """
        Update the position of the bird
        """

        s = min(16, self.vel + 1.5)
        self.vel = min(16, self.vel + 3)

        self.y = self.y + s
        if s < 0:
            if self.tilt < self.MAX_ROTATION:
                self.tilt += self.ROT_VEL
        else:
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL
    
    def draw(self, window):
        """
        Draw the bird on the window
        """

        self.cnt += 1

        if self.cnt < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.cnt < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.cnt < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.cnt < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.cnt == self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.cnt = 0

        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.cnt = self.ANIMATION_TIME*2

        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center=self.img.get_rect(topleft=(self.x, self.y)).center)
        window.blit(rotated_image, new_rect.topleft)

    def get_mask(self):
        """
        Returns: pygame.mask: mask of the bird
        """
        
        return pygame.mask.from_surface(self.img)

In [2]:
class Pipe:
    """
    The Pipe Class
    """

    def __init__(self, x, GAP = 200, VEL = 5):
        """
        A Constructor for the Pipe Class

        x: int: x-coordinate of the pipe
        GAP: int: gap between the two pipes
        VEL: int: velocity of the pipe
        """
        self.x = x
        self.height = 0
        self.GAP = GAP
        self.VEL = VEL

        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG, False, True)
        self.PIPE_BOTTOM = PIPE_IMG

        self.passed = False
        self.set_height()

    def set_height(self):
        """
        Set the height of the pipe
        """

        self.height = random.randrange(50, 400)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP

    def move(self):
        """
        Update the position of the pipe
        """

        self.x -= self.VEL

    def draw(self, window):
        """
        Draw the pipe on the window

        window: pygame.Surface: the window to draw the pipe on
        """

        window.blit(self.PIPE_TOP, (self.x, self.top))
        window.blit(self.PIPE_BOTTOM, (self.x, self.bottom))

    def collide(self, bird):
        """
        Check if the bird has collided with the pipe

        bird: Bird: the bird object

        Returns: bool: True if the bird has collided with the pipe, False otherwise
        """

        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)

        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))

        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(top_mask, top_offset)

        if b_point or t_point:
            return True
        return False

In [5]:
class Base:
    """
    Defining the Base Class
    """

    WIDTH = BASE_IMG.get_width()
    IMG = BASE_IMG

    def __init__(self, y, VEL = 5):
        """
        A Constructor for the Base Class

        y: int: y-coordinate of the base
        VEL: int: velocity of the base
        """
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH
        self.VEL = VEL

    def move(self):
        """
        Update the position of the base
        """

        self.x1 -= self.VEL
        self.x2 -= self.VEL

        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH
        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH

    def draw(self, window):
        """
        Draw the base on the window

        window: pygame.Surface: window to draw the base on
        """
        window.blit(self.IMG, (self.x1, self.y))
        window.blit(self.IMG, (self.x2, self.y))

In [6]:
"""
Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 150
RESTART_Y = 300
BASE = 650

class FlappyBirdGame:
    """
    The Flappy Bird Game Class
    """

    def __init__(self, window, base, pipes, bird):
        """
        A Constructor for the Flappy Bird Game Class

        window: pygame.Surface: the window to display the game
        base: Base: the base object
        pipes: list: list of pipes
        bird: Bird: the bird object
        """

        self.window = window
        self.base = base
        self.pipes = pipes
        self.bird = bird

        self.clock = pygame.time.Clock()
        self.running = True
        self.score = 0

    def gameplay(self):
        """
        The main gameplay function
        """

        game_over = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.KEYDOWN:
                    if game_over and event.key == pygame.K_SPACE:
                        self.bird = Bird(RESTART_X, RESTART_Y)
                        self.pipes = [Pipe(WIN_WIDTH + 50)]
                        self.base = Base(BASE)
                        self.score = 0
                        game_over = False
                    if event.key == pygame.K_SPACE:
                        self.bird.jump()

            if not game_over:
                self.bird.move()
                for pipe in self.pipes:
                    pipe.move()
                    if pipe.collide(self.bird):
                        game_over = True
                    if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                        self.pipes.remove(pipe)
                    if not pipe.passed and pipe.x < self.bird.x:
                        pipe.passed = True
                        self.score += 1
                        self.pipes.append(Pipe(WIN_WIDTH + 50))
                self.base.move()

                if self.bird.y + self.bird.img.get_height() >= BASE or self.bird.y < 0:
                    game_over = True

            if game_over:
                self.Game_Over()
            else:
                self.display()
        pygame.quit()

    def display(self):
        """
        Display the game on the window
        """

        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        self.bird.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        pygame.display.update()

    def Game_Over(self):
        """
        Display the Game Over Screen
        """
        
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        self.bird.draw(self.window)
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 50).render("Game Over", 1, (255, 255, 255))
        self.window.blit(text, (WIN_WIDTH//2 - text.get_width()//2, WIN_HEIGHT//2 - text.get_height()//2))
        pygame.display.update()

### Defining Game Structure

In [12]:
def main():
    bird = Bird(RESTART_X, RESTART_Y)
    pipes = [Pipe(300)]
    base = Base(BASE)
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    pygame.font.init()
    game = FlappyBirdGame(window, base, pipes, bird)
    game.gameplay()

main()

### Model Creation and Training

In [9]:
"""
AI Game Structure and Functionality
"""
pygame.font.init()
RESTART_X = 150
RESTART_Y = 300
BASE = 650

class FlappyBirdGameAI:
    """
    The Flappy Bird Game Class
    """

    def __init__(self, window, base, pipes, birds, genes, nets):
        """
        A Constructor for the Flappy Bird Game Class

        window: pygame.Surface: the window to display the game
        base: Base: the base object
        pipes: list: list of pipes
        birds: list: list of bird objects
        genes: list: list of genes
        nets: list: list of neural networks
        """

        self.window = window
        self.base = base
        self.pipes = pipes

        self.birds = birds
        self.genes = genes
        self.nets = nets

        self.clock = pygame.time.Clock()
        self.running = True
        self.gen = 0
        self.score = 0

    def gameplay(self):
        """
        The main gameplay function
        """

        add_pipe = False
        while self.running:
            self.clock.tick(30)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                if event.type == pygame.K_ESCAPE:
                    self.running = False
                    raise neat.CompleteGenome(self.birds[0], self.genes[0], self.nets[0])

            pipe_ind = 0
            if len(self.birds) > 0:
                if len(self.pipes) > 1 and self.birds[0].x > self.pipes[0].x + self.pipes[0].PIPE_TOP.get_width():
                    pipe_ind = 1
            else:
                self.running = False
                break

            for x, bird in enumerate(self.birds):
                self.genes[x].fitness += 0.1
                bird.move()
                output = self.nets[x].activate((bird.y, abs(bird.y - self.pipes[pipe_ind].height), abs(bird.y - self.pipes[pipe_ind].bottom)))
                if output[0] > 0.5:
                    bird.jump()

            for pipe in self.pipes:
                pipe.move()
                for i, bird in enumerate(self.birds):
                    if pipe.collide(bird):
                        self.genes[i].fitness -= 1
                        self.birds.pop(i)
                        self.nets.pop(i)
                        self.genes.pop(i)

                    if not pipe.passed and pipe.x < bird.x:
                        pipe.passed = True
                        add_pipe = True
                        
                if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                    self.pipes.remove(pipe)

            for i, bird in enumerate(self.birds):
                if bird.y + bird.img.get_height() >= BASE or bird.y < 0:
                    self.birds.pop(i)
                    self.nets.pop(i)
                    self.genes.pop(i)

            if add_pipe:
                self.score += 1
                for gene in self.genes:
                    gene.fitness += 5
                pipe = Pipe(WIN_WIDTH, GAP = 200 - 1.5*self.score, VEL = 5 + 0.05*self.score)
                self.base.VEL = pipe.VEL
                self.pipes.append(pipe)
                add_pipe = False

            self.base.move()

            self.display(DRAW_LINES = True)
        pygame.quit()

    def display(self, DRAW_LINES = False):
        """
        Display the game on the window

        DRAW_LINES: bool: True if the lines are to be drawn, False otherwise
        """
        
        self.window.blit(BG_IMG, (0, 0))
        for pipe in self.pipes:
            pipe.draw(self.window)
        self.base.draw(self.window)
        for bird in self.birds:
            bird.draw(self.window)

        if DRAW_LINES and len(self.birds) > 0:
            pipe_ind = 0
            if len(self.pipes) > 1 and self.birds[0].x > self.pipes[0].x + self.pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1
            for bird in self.birds:
                pygame.draw.line(self.window, (255, 0, 0), (bird.x + bird.img.get_width()//2, bird.y + bird.img.get_height()//2), (self.pipes[pipe_ind].x + self.pipes[pipe_ind].PIPE_TOP.get_width()//2, self.pipes[pipe_ind].height), 5)
                pygame.draw.line(self.window, (255, 0, 0), (bird.x + bird.img.get_width()//2, bird.y + bird.img.get_height()//2), (self.pipes[pipe_ind].x + self.pipes[pipe_ind].PIPE_BOTTOM.get_width()//2, self.pipes[pipe_ind].bottom), 5)
        
        text = pygame.font.SysFont("norwester", 25).render(f"Generation: {self.gen}", 1, (255, 255, 255))
        self.window.blit(text, (10, 10))
        text = pygame.font.SysFont("norwester", 25).render(f"Score: {self.score}", 1, (255, 255, 255))
        self.window.blit(text, (10, 40))
        text = pygame.font.SysFont("norwester", 25).render(f"Alive: {len(self.birds)}", 1, (255, 255, 255))
        self.window.blit(text, (10, 70))

        pygame.display.update()

In [10]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_FBAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
generation = 0

def main(genomes, config):
    global generation

    birds = []
    genes = []
    nets = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(RESTART_X, RESTART_Y))
        g.fitness = 0
        genes.append(g)

    pipes = [Pipe(300)]
    base = Base(BASE)
    window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

    try:
        pygame.font.init()
        game = FlappyBirdGameAI(window, base, pipes, birds, genes, nets)
        game.gen = generation
        game.gameplay()
    except neat.CompleteGenome as e:
        raise e

    generation += 1

try:
    winner = population.run(main, 50)
except neat.CompleteGenome as e:
    winner = e.genome

with open("Models/FlappyBirdAI.pkl", "wb") as genome_file:
    pickle.dump(winner, genome_file)
print(f"The Best Genome:\n{winner}")


 ****** Running generation 0 ****** 

Population's average fitness: 1.86500 stdev: 2.32879
Best fitness: 11.90000 - size: (1, 3) - species 1 - id 9
Average adjusted fitness: 0.079
Mean genetic distance 1.288, standard deviation 0.718
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20     11.9    0.079     0
Total extinctions: 0
Generation time: 2.736 sec

 ****** Running generation 1 ****** 

Population's average fitness: 1.51000 stdev: 0.38458
Best fitness: 2.30000 - size: (1, 3) - species 1 - id 25
Average adjusted fitness: 0.342
Mean genetic distance 1.105, standard deviation 0.474
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20      2.3    0.342     1
Total extinctions: 0
Generation time: 0.912 sec (1.824 average)

 ****** Running generation 2 ****** 

Population's average fitness: 2.20000 stdev: 2.24

### Genome Loading

In [40]:
with open("FlappyBirdAI.pkl", "rb") as genome_file:
    genome = pickle.load(genome_file)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "Configs/config_FBAI.txt")
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

model = neat.nn.FeedForwardNetwork.create(genome, config)

In [45]:
print(model.activate((30, 100, 100)))

[-1.0]
